In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC


In [4]:
import os

# Get the current working directory
current_directory = os.getcwd()
print(current_directory)

/content


In [5]:
from typing_extensions import assert_type
#getting files
gender = pd.read_csv('gender_submission.csv')
X_test = pd.read_csv('test.csv')
df = pd.read_csv('train.csv')


df['train_test'] = 1
X_test['train_test'] =0
X_test['Survived'] = np.NaN
all_data = pd.concat([df, X_test])

In [6]:
#check nulls values in data
missing_values = df.isnull().sum()
print(missing_values)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
train_test       0
dtype: int64


In [7]:
#filling nulls with 0/ S / mean
all_data['Cabin'] = all_data['Cabin'].fillna(0)
all_data['Embarked'] = all_data['Embarked'].fillna('S')
all_data['Age'] = all_data['Age'].fillna(X_test['Age'].mean())
all_data['Fare'] = all_data['Fare'].fillna(X_test['Fare'].median())


In [8]:
#changing type to string
all_data['Cabin'] =all_data['Cabin'].astype(str)

In [9]:
#data modelling
# binary sex , ports to numbers, titles to numbers, splitting cabin
all_data['Sex_binary'] = all_data['Sex'].apply(lambda x:1 if x=='male' else 0)
all_data['Port'] = all_data['Embarked'].apply(lambda x:1 if x=='S' else(2 if x=='Q' else 0))
all_data['Title'] = all_data['Name'].apply(lambda x:x.split(',')[1].split('.')[0].strip())
all_data['Bo_Title'] = all_data['Title'].apply(lambda x:1 if x=='Mr' else(2 if x=='Miss' or x=='Lady' or x=='Mrs' or x=='Mlle' or x=='Mme' else 0))
all_data['Cabin_splitted'] = all_data['Cabin'].apply(lambda x:x.split(' ')[0] if x else 0)
all_data['Ticket_splitted'] = all_data['Ticket'].apply(lambda x:x.split(' ')[0] if x else 0)


In [10]:
all_data['Title']

,Title
0,Mr
1,Mrs
2,Miss
3,Mrs
4,Mr
...,...
413,Mr
414,Dona
415,Mr
416,Mr


In [11]:
#exporting level of ship from cabin
import string

all_data['Cabin_splitted'] = all_data['Cabin_splitted'].fillna(0)
def letter_to_number(letter):
    if letter and letter.isalpha():
        return string.ascii_uppercase.index(letter.upper()) + 1
    else:
        return 0

level = all_data['Cabin_splitted'].apply(lambda x: letter_to_number(x[0] if x else 0))
all_data['level'] = level


In [12]:
#exporting level of ship from cabin
import string

all_data['Ticket_splitted'] = all_data['Ticket_splitted'].fillna(0)
def letter_to_number(letter):
    if letter and letter.isalpha():
        return string.ascii_uppercase.index(letter.upper()) + 1
    else:
        return 0

Ticket_number = all_data['Ticket_splitted'].apply(lambda x: letter_to_number(x[0] if x else 0))

all_data['Ticket'] = Ticket_number

In [13]:
all_data['Ticket']

,Ticket
0,1
1,16
2,19
3,0
4,0
...,...
413,1
414,16
415,19
416,0


In [14]:
X_train1 = all_data[all_data.train_test == 1].drop(['train_test'], axis =1)
X_train = X_train1[['PassengerId','Age', 'SibSp','Ticket', 'Parch','Sex_binary', 'Fare','Pclass','Port','Bo_Title','level']].astype(str)
X_test1 = all_data[all_data.train_test == 0].drop(['train_test'], axis =1)
X_test = X_test1[['PassengerId','Age', 'SibSp','Ticket', 'Parch','Sex_binary','Fare', 'Pclass','Port','Bo_Title','level']].astype(str)


In [15]:
X_test.isna().sum()

,0
PassengerId,0
Age,0
SibSp,0
Ticket,0
Parch,0
Sex_binary,0
Fare,0
Pclass,0
Port,0
Bo_Title,0


In [16]:
#making list of features
Y_train = X_train1['Survived'] == 1
y_test = X_test1['Survived']

In [17]:
#importing ml models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb

In [18]:
y_test

,Survived
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
413,NaN
414,NaN
415,NaN
416,NaN


In [19]:
model1 = HistGradientBoostingClassifier()
model1.fit(X_train, Y_train)

HistGradientBoostingClassifier()

In [20]:
model3 = DecisionTreeClassifier()
model3.fit(X_train, Y_train)

DecisionTreeClassifier()

In [21]:
X_train.dtypes

,0
PassengerId,object
Age,object
SibSp,object
Ticket,object
Parch,object
Sex_binary,object
Fare,object
Pclass,object
Port,object
Bo_Title,object


In [22]:
X_train = X_train[['PassengerId','Age', 'SibSp', 'Parch', 'Sex_binary', 'Fare', 'Pclass', 'Port', 'Bo_Title', 'level']].astype(float)
X_test = X_test[['PassengerId','Age', 'SibSp', 'Parch', 'Sex_binary', 'Fare', 'Pclass', 'Port', 'Bo_Title', 'level']].astype(float)

In [23]:
model5 = xgb.XGBClassifier()
model5.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [24]:
model6 = KNeighborsClassifier(n_neighbors=15)
model6.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=15)

In [25]:
from sklearn.ensemble import RandomForestClassifier
model7 = RandomForestClassifier(n_estimators=5000, random_state=42)
model7.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=5000, random_state=42)

In [26]:
ensemble = VotingClassifier(estimators=[('hg1', model1),('dt1', model3),('xg1',model5), ('kn', model6),('rf', model7)], voting='soft')

In [27]:
ensemble.fit(X_train, Y_train )

VotingClassifier(estimators=[('hg1', HistGradientBoostingClassifier()),
                             ('dt1', DecisionTreeClassifier()),
                             ('xg1',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow...
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('kn', KNeighborsClassifier(n_neighbors=15)),
                             ('rf',
                              RandomForestClassifier(n_estimators=5000,
                                                     random_state=42))],
                 voting='soft')

In [28]:
y_pred = ensemble.predict(X_test)
y_pred

array([False, False, False, False, False, False,  True, False,  True,
       False, False, False,  True, False,  True,  True, False, False,
       False, False, False,  True,  True, False,  True, False,  True,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False,  True,  True,
       False, False, False,  True, False, False, False,  True,  True,
       False, False, False, False, False,  True, False, False, False,
        True,  True,  True, False, False,  True,  True, False, False,
       False,  True,  True, False, False,  True, False, False,  True,
       False, False, False, False, False, False, False, False,  True,
       False, False,  True, False, False, False,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True, False, False,
       False, False,  True,  True, False,  True, False, False,  True,
       False, False,

In [29]:
#from sklearn.metrics import accuracy_score

# Calculate predictions on the test set using the ensemble
#y_pred = ensemble.predict(X_train)

# Calculate accuracy using the true labels (Y_test) and predicted labels
#accuracy = accuracy_score(X_train1, y_pred)  # Replace X_test with Y_test
#print("Test Accuracy:", accuracy)  # Changed from "Training Accuracy"

In [30]:
submission = pd.DataFrame({
    'PassengerId': X_test1['PassengerId'],
    'Survived': y_pred.astype(int)  # Convert predictions to integers
})

In [31]:
submission.to_csv('submission.csv', index=False)

In [32]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
